In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.stats import skew
from scipy.special import boxcox1p
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error,  mean_absolute_error, mean_absolute_percentage_error,r2_score
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!pip install -q shap
import shap

In [3]:
properties=pd.read_csv("ames.csv")

#Null data and summary

In [ ]:
head = properties.head()
shape = properties.shape
describe = properties.describe().T
print(shape)
print(head)

In [ ]:
properties.describe().T

In [ ]:
#Tells how many values are present in each column and how many %of zeroes are there
def data_summaries (df):
    for col in df.columns:
        print ("__%s__" % col,'\n')
        if df[col].dtype == np.object:
            print (df[col].value_counts())
        elif len(df[df[col] == 0]) >= 1465:
            print ('% zeros (HIGH!): ', round((len(df[df[col] == 0])/2930)*100),\
                   '\n','no. zeros: ',len(df[df[col]==0]))
        else:
            print ('% zeros: ', round((len(df[df[col] == 0])/2930)*100))
        print ('\n')
    
data_summaries(properties)

In [ ]:
properties.columns[properties.isnull().any()]

In [ ]:
#Tells the null value present in the dataset in descending order
null_data = []
for item in properties.isnull().sum().items():
    null_data.append([item[0], int(item[1])])

null_data = np.array(null_data, dtype=np.object)
null_data = null_data[np.argsort(null_data[:, 1])][::-1]
null_data

In [9]:
# this handles the null values in the dataset
# if the column is float, it imputes with the mean
# if the column is categorical, then depending on the column type
# it imputes with either the mode or a seperate '0'`value
# if it is expected that nulis the default, then it imputes using the '0'`- e.g. null means no pool
# whereas if it is expected that null means a missing value, then it is replaced with the mode

impute_with_na = ['Pool.QC', 
                 'Misc.Feature',
                 'Alley',
                 'Fence',
                 'Fireplace.Qu',
                 ]

impute_with_mode = ['Garage.Qual',
                    'Garage.Cond',
                    'Garage.Finish',
                    'Garage.Type',
                    'Bsmt.Exposure',
                    'BsmtFin.Type.2',
                    'BsmtFin.Type.1',
                    'Bsmt.Cond',
                    'Bsmt.Qual',
                    'Mas.Vnr.Type',
                    'Electrical',
                    ]
properties_processed = properties.copy()
for row in null_data:
  if row[1]>0:
    if properties[row[0]].dtype==float:
        properties_processed[row[0]] = properties_processed[row[0]].fillna(properties_processed[row[0]].mean())
        continue

    if row[0] in impute_with_na:
        properties_processed[row[0]].loc[properties_processed[row[0]].isnull()]='Na'
        continue
    if row[0] in impute_with_mode:
        properties_processed[row[0]].loc[properties_processed[row[0]].isnull()]=properties_processed[row[0]].mode().values[0]
        continue
    raise f"Column {row[0]} not handled"
    
    print("\n\n")

In [ ]:
# Checks if there are any null values present 
properties_processed.columns[properties_processed.isnull().any()]

In [ ]:
# All nominal and ordinal features with their values
cat = properties_processed.select_dtypes(include=['object'])
for (colname,colval) in cat.iteritems():
    print(colname, colval.unique())

In [12]:
col_numerical = [col for col in properties_processed.columns if properties_processed[col].dtype != np.object]
col_cat = [col for col in properties_processed.columns if properties_processed[col].dtype == np.object]

#EDA

In [ ]:
#To check for skewness of the data in ordinal/nominal columns

nominal_stats = {'column': [], 'most_frequent_value': [], 'percentage_occurrence':[], 'column_status': []}

threshold = 90.0 # threshold value for considering the column to be severely skewed towards the mode value

for col in col_cat:
    nominal_stats['column'].append(col)
    mode = properties_processed.loc[:,col].mode()[0]
    nominal_stats['most_frequent_value'].append(mode)
    proportion = np.round(properties_processed.loc[properties_processed.loc[:,col]==properties_processed.loc[:,col].mode()[0],:].shape[0] / properties_processed.shape[0] * 100, 1)
    nominal_stats['percentage_occurrence'].append(proportion)
    nominal_stats['column_status'].append('' if proportion < threshold else 'Severely skewed')

nominal_stats_df = pd.DataFrame(nominal_stats)
nominal_stats_df

In [ ]:
#To check for skewness of the data in cont/discrete columns

nominal_stats = {'column': [], 'most_frequent_value': [], 'percentage_occurrence':[], 'column_status': []}

threshold = 90.0 # threshold value for considering the column to be severely skewed towards the mode value

for col in col_numerical:
    nominal_stats['column'].append(col)
    mode = properties_processed.loc[:,col].mode()[0]
    nominal_stats['most_frequent_value'].append(mode)
    proportion = np.round(properties_processed.loc[properties_processed.loc[:,col]==properties_processed.loc[:,col].mode()[0],:].shape[0] / properties_processed.shape[0] * 100, 1)
    nominal_stats['percentage_occurrence'].append(proportion)
    nominal_stats['column_status'].append('' if proportion < threshold else 'Severely skewed')

nominal_stats_df = pd.DataFrame(nominal_stats)
nominal_stats_df

In [15]:
# histogram subplots
def subplot_histograms(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(20,nrows*4)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        ax[i].hist(dataframe[column],bins=15)
        ax[i].set_title(f'{column} distribution',fontsize=17)
        ax[i].tick_params(labelsize=15)
        ax[i].set_xlabel(column, fontsize=15)
        ax[i].set_ylabel(f'{column} values', fontsize=15)
    plt.tight_layout()
    
# scatterplot subplots
def subplot_scatter(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(20, nrows*4)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        sns.regplot(y=dataframe.SalePrice, x=dataframe[column],ax=ax[i], 
                    scatter_kws={'facecolors':'lightgreen','edgecolor':'lightgreen'},
                    line_kws = {'color':'red'})
        ax[i].set_title(f'{column} vs SalePrice',fontsize=15)  
        ax[i].tick_params(labelsize=15)
        ax[i].set_xlabel(column, fontsize=15)
        ax[i].set_ylabel('SalePrice', fontsize=15) 
    plt.tight_layout()
    
# boxplot subplots
def subplot_box(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(20, nrows*4)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        sns.boxplot(x = dataframe[column], y = dataframe.SalePrice, width = 0.5, ax = ax[i], color='lightgreen')
        ax[i].set_title(column,fontsize=15)  
        ax[i].tick_params(labelsize=15)
        ax[i].set_xlabel(column, fontsize=15)
        ax[i].set_ylabel('SalePrice', fontsize=15)
    plt.tight_layout()
    
# distribution plots (histogram, boxplot, probplot)
def dist_plots(df, list_of_columns):
    nrows = len(list_of_columns)
    fig, ax = plt.subplots(nrows = nrows, ncols = 3, figsize=(20, nrows*4))
    ax = ax.ravel()
    for i, col in enumerate(list_of_columns):
        sns.distplot(df[col], ax = ax[i*3-3], fit = stats.norm)
        ax[i*3-3].set_title(f'{col} distribution plot',fontsize=15)
        ax[i*3-3].tick_params(labelsize=15)
        ax[i*3-3].set_xlabel(col, fontsize=15)
        
        sns.boxplot(df[col], width = 0.2, ax = ax[i*3-2])
        ax[i*3-2].set_title(f'{col} box plot',fontsize=15)
        ax[i*3-2].tick_params(labelsize=15)
        ax[i*3-2].set_xlabel(col, fontsize=15)
        
        stats.probplot(df[col], plot = ax[i*3-1])
        ax[i*3-1].set_title(f'{col} probability plot', fontsize=15)
        ax[i*3-1].tick_params(labelsize=15)
        ax[i*3-1].set_xlabel(col, fontsize=15)
    plt.tight_layout()

In [ ]:
dist_plots(properties_processed,col_numerical)

In [ ]:
subplot_scatter(properties_processed,col_numerical)

In [ ]:
subplot_histograms(properties_processed,col_cat)

In [ ]:
subplot_box(properties_processed, col_cat)

In [ ]:
# order columns
properties_processed = properties_processed[properties_processed.columns.sort_values()]

# plot heatmap
mask = np.zeros_like(properties_processed.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)]= True

f, ax = plt.subplots(figsize=(40, 40))
sns.heatmap(properties_processed.corr(), 
            annot=True,
            mask = mask,
            square=True,
            vmin = -1,
            vmax = 1,
            linewidth=0.2,
            cbar_kws = {'shrink':0.75},
            cmap=sns.color_palette("husl", 9),
            annot_kws={'size': 13})
ax.tick_params(labelsize=10)
plt.tight_layout()

In [21]:
# One Hot Encoding
properties_processed_ohe = pd.get_dummies(properties_processed)

In [ ]:
# get index of columns with 10 highest corrcoef with respect to saleprice
correlation_coefficients = properties_processed_ohe.corr().nlargest(10, 'SalePrice').index

# create heatmap
mask = np.zeros_like(properties_processed_ohe[correlation_coefficients].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)]= True

f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(properties_processed_ohe[correlation_coefficients].corr(), 
            annot=True, 
            square= True, 
            mask = mask,
            cmap=sns.color_palette("husl", 9),
            annot_kws={'size': 13},
            cbar_kws={"shrink": 0.75},
            linewidth = 0.1,
            yticklabels=correlation_coefficients.values, 
            xticklabels=correlation_coefficients.values,
            vmax = 1,
            vmin = -1)
ax.set_xlim(0,10)
ax.set_ylim(0,10)
ax.tick_params(labelsize=12)
plt.title('Features with highest correlation coefficients', fontsize=19)
plt.tight_layout()

In [ ]:
print("Top 10 Values:")
numeric_data = properties_processed.select_dtypes(include=[np.number])
corr = numeric_data.corr()
print (corr['SalePrice'].sort_values(ascending=False)[:10], '\n')
print ('----------------------')
print("Last 10 values:")
print (corr['SalePrice'].sort_values(ascending=False)[-10:])

In [ ]:
sns.pairplot(properties_processed_ohe[correlation_coefficients],size = 2 ,kind ='scatter',diag_kind='kde',corner=True)
plt.tight_layout()

#Feature Engineering 

In [25]:
# Converting ordinal columns with type object to int by assigning ranks. Using single mapper for all ordinal variables with their codes

ordinal_mapper = {'Na': 0, 'Po': 1, 'Fa': 2, 'TA': 3,'Gd': 4,'Ex': 5 }
properties_processed['Exter.Qual']    = properties_processed['Exter.Qual'].replace(ordinal_mapper)
properties_processed['Exter.Cond']    = properties_processed['Exter.Cond'].replace(ordinal_mapper)
properties_processed['Bsmt.Qual']     = properties_processed['Bsmt.Qual'].replace(ordinal_mapper)
properties_processed['Bsmt.Cond']     = properties_processed['Bsmt.Cond'].replace(ordinal_mapper)
properties_processed['Garage.Qual']   = properties_processed['Garage.Qual'].replace(ordinal_mapper)
properties_processed['Garage.Cond']   = properties_processed['Garage.Cond'].replace(ordinal_mapper)
properties_processed['Heating.QC']    = properties_processed['Heating.QC'].replace(ordinal_mapper)
properties_processed['Kitchen.Qual']  = properties_processed['Kitchen.Qual'].replace(ordinal_mapper)
properties_processed['Fireplace.Qu']  = properties_processed['Fireplace.Qu'].replace(ordinal_mapper)
properties_processed['Pool.QC']       = properties_processed['Pool.QC'].replace(ordinal_mapper)

properties_processed['Functional'] = properties_processed['Functional'].replace({'Sal': 8, 'Sev': 1, 'Maj2': 2, 'Maj1': 3,'Mod': 4,'Min2': 5, 'Min1': 6 ,'Typ': 7 })

properties_processed['Central.Air'] = properties_processed['Central.Air'].map({'N': 1,'Y': 2})

properties_processed['Fence'] = properties_processed['Fence'].map({'Na':0, 'MnPrv':1, 'GdPrv':2, 'GdWo':3, 'MnWw':4})

properties_processed['MS.Zoning'] = properties_processed['MS.Zoning'].map({'RL':1, 'RH':2, 'FV':3, 'RM':4, 'C (all)':5, 'I (all)':6, 'A (agr)':7})

properties_processed['Alley'] = properties_processed['Alley'].map({'Na':0 ,'Pave':1 ,'Grvl':2})

properties_processed['Street'] = properties_processed['Street'].map({'Na':0 ,'Pave':1 ,'Grvl':2})

properties_processed['Lot.Shape'] = properties_processed['Lot.Shape'].map({'IR1':1, 'Reg':2 ,'IR2':3, 'IR3':4})

properties_processed['Land.Contour'] = properties_processed['Land.Contour'].map({'Lvl':1, 'HLS':2, 'Bnk':3, 'Low':4})

properties_processed['Utilities'] = properties_processed['Utilities'].map({'AllPub':1, 'NoSewr':2, 'NoSeWa':3})

properties_processed['Lot.Config'] = properties_processed['Lot.Config'].map({'Corner':1, 'Inside':2, 'CulDSac':3, 'FR2':4, 'FR3':5})

properties_processed['Land.Slope'] = properties_processed['Land.Slope'].map({'Gtl':1, 'Mod':2, 'Sev':3})

properties_processed['Condition.1'] = properties_processed['Condition.1'].map({'Norm':1, 'Feedr':2, 'PosN':3, 'RRNe':9, 'RRAe':5, 'Artery':6, 'PosA':7, 'RRAn':8, 'RRNn':4})

properties_processed['Condition.2'] = properties_processed['Condition.2'].map({'Norm':1, 'Feedr':2, 'PosN':3,'RRAe':5, 'Artery':6, 'PosA':7, 'RRAn':8, 'RRNn':4})

properties_processed['Bldg.Type'] = properties_processed['Bldg.Type'].map({'1Fam':1, 'TwnhsE':2, 'Twnhs':3, 'Duplex':4, '2fmCon':5})

properties_processed['House.Style'] = properties_processed['House.Style'].map({'1Story':1, '2Story':2, '1.5Fin':3, 'SFoyer':4, 'SLvl':5, '2.5Unf':6, '1.5Unf':7, '2.5Fin':8})

properties_processed['Roof.Style'] = properties_processed['Roof.Style'].map({'Hip':1, 'Gable':2, 'Mansard':3, 'Gambrel':4, 'Shed':5, 'Flat':6})

properties_processed['Roof.Matl'] = properties_processed['Roof.Matl'].map({'CompShg':1, 'WdShake':2, 'Tar&Grv':3, 'WdShngl':4, 'Membran':5, 'ClyTile':6, 'Roll':7, 'Metal':8})

properties_processed['Mas.Vnr.Type'] = properties_processed['Mas.Vnr.Type'].map({'Stone':1, 'None':2, 'BrkFace':3, 'BrkCmn':4, 'CBlock':5})

properties_processed['Foundation'] = properties_processed['Foundation'].map({'CBlock':1, 'PConc':2, 'Wood':3, 'BrkTil':4, 'Slab':5, 'Stone':6})

properties_processed['Heating'] = properties_processed['Heating'].map({'GasA':1, 'GasW':2, 'Grav':3, 'Wall':4, 'Floor':5, 'OthW':6})

properties_processed['Electrical'] = properties_processed['Electrical'].map({'SBrkr':1, 'FuseA':2, 'FuseF':3, 'FuseP':4, 'Mix':5})

properties_processed['Bsmt.Exposure'] = properties_processed['Bsmt.Exposure'].map({'Gd':1, 'No':2, 'Mn':3, 'Av':4})

properties_processed['BsmtFin.Type.1'] = properties_processed['BsmtFin.Type.1'].map({'BLQ':1, 'Rec':2, 'ALQ':3, 'GLQ':4, 'Unf':5, 'LwQ':6})

properties_processed['BsmtFin.Type.2'] = properties_processed['BsmtFin.Type.2'].map({'BLQ':1, 'Rec':2, 'ALQ':3, 'GLQ':4, 'Unf':5, 'LwQ':6})

properties_processed['Garage.Type'] = properties_processed['Garage.Type'].map({'Attchd':1, 'BuiltIn':2, 'Basment':3, 'Detchd':4, 'CarPort':5, '2Types':6})

properties_processed['Garage.Finish'] = properties_processed['Garage.Finish'].map({'Fin':1, 'Unf':2, 'RFn':3})

properties_processed['Paved.Drive'] = properties_processed['Paved.Drive'].map({'P':1, 'Y':2, 'N':3})

properties_processed['Misc.Feature'] = properties_processed['Misc.Feature'].map({'Na':0 ,'Gar2':1, 'Shed':2, 'Othr':3, 'Elev':4, 'TenC':5})

properties_processed['Sale.Type'] = properties_processed['Sale.Type'].map({'WD ':1, 'New':2, 'COD':3, 'ConLI':4, 'Con':5, 'ConLD':6, 'Oth':7, 'ConLw':8, 'CWD':9, 'VWD':10})

properties_processed['Sale.Condition'] = properties_processed['Sale.Condition'].map({'Normal':1, 'Partial':2, 'Family':3, 'Abnorml':4, 'Alloca':5, 'AdjLand':6})

properties_processed['Exterior.1st'] = properties_processed['Exterior.1st'].map({'BrkFace':1, 'VinylSd':2,'Wd Sdng':3, 'CemntBd':4, 'HdBoard':5, 'Plywood':6, 'MetalSd':7,
                                                                                     'AsbShng':8, 'WdShing':15, 'Stucco':9, 'AsphShn':10, 'BrkComm':16, 'CBlock':11, 'PreCast':12,
                                                                                     'Stone':13, 'ImStucc':14})

properties_processed['Exterior.2nd'] = properties_processed['Exterior.2nd'].map({'BrkFace':1, 'VinylSd':2,'Wd Sdng':3, 'CmentBd':4, 'HdBoard':5, 'Plywood':6, 'MetalSd':7,
                                                                                     'AsbShng':8, 'Wd Shng':15, 'Stucco':9, 'AsphShn':10, 'Brk Cmn':16, 'CBlock':11, 'PreCast':12,
                                                                                     'Stone':13, 'ImStucc':14, 'Other':17})

In [ ]:
properties_processed['SalePrice'].groupby(properties_processed['Neighborhood']).median().sort_values().plot(kind='bar')

In [27]:
#Depending on the above graph we can classify what region should we put the neighbourhoods in

properties_processed['Neighborhood'] = properties_processed['Neighborhood'].map({'MeadowV':1, 'BrDale':1, 'IDOTRR':1,     
'OldTown':2, 'Edwards':2, 'BrkSide':2, 'Blueste':2, 'Sawyer':2, 'SWISU':2, 'Landmrk':2, 'NAmes':2, 'NPkVill':2, 'Mitchel':2, 
'SawyerW':3, 'NWAmes':3, 'Gilbert':3, 'Blmngtn':3, 'ClearCr':3, 'Greens':3, 'CollgCr':3, 'Crawfor':3,                    
'Somerst':4, 'Timber':4, 'Veenker':4, 'GrnHill':4, 'NoRidge':4, 'NridgHt':4, 'StoneBr':4})  

In [ ]:
properties_processed['SalePrice'].groupby(properties_processed['Neighborhood']).median().sort_values().plot(kind='bar')

In [29]:
#cross check to see if any object features are present 
cat = properties_processed.select_dtypes(include=['object'])
for (colname,colval) in cat.iteritems():
    print(colname, colval.unique())

In [ ]:
properties_processed.columns

In [31]:
#Drop columnns and outliers

def process(dataframe):
  df = dataframe.copy()

  # function to drop columns
  def dropcol(df, dropscol):
   df.drop(dropscol, axis = 1, inplace = True)

  # columns with >80% single category or >80% zeros, plus pid and order columns
  dropcols = ['Order','PID','Street','Roof.Matl','Alley','Utilities','Condition.2','Heating','Garage.Cond','Misc.Val','Misc.Feature','X3Ssn.Porch',
              'Bsmt.Half.Bath','Low.Qual.Fin.SF','Land.Slope','Garage.Yr.Blt','Garage.Qual',
              ]
  dropcol(df, dropcols)


  #Outliers
  df.drop(df[df['Lot.Frontage'] > 250].index, inplace = True)
  df.drop(df[df['Lot.Area'] > 100000].index, inplace = True)
  df.drop(df[df['Gr.Liv.Area'] > 4500].index, inplace = True)
  df.drop(df[df['X1st.Flr.SF'] > 4000].index, inplace = True)
  df.drop(df[df['Total.Bsmt.SF'] > 4000].index, inplace = True)
  df.drop(df[df['BsmtFin.SF.1'] > 3000].index, inplace = True)

  df=df.reset_index(drop=True)
  return df

In [32]:
fully_properties_processed=process(properties_processed)
fully_properties_processed_ohe = pd.get_dummies(fully_properties_processed)

In [33]:
def rmse(ytest, ypred):
    return np.sqrt(mean_squared_error(ytest, ypred))

# Random Forest 

In [34]:
def calculate_RSS(X_rss, y_rss, attr, s):
    d_1 = y_rss[X_rss[:, attr]<=s]
    d_2 = y_rss[X_rss[:, attr]>s]
    
    if len(d_1)==0:
        rss_1 = 0
    else:
        rss_1 = ((d_1 - d_1.mean())**2).sum()
        
    if len(d_2)==0:
        rss_2 = 0
    else:
        rss_2 = ((d_2 - d_2.mean())**2).sum() 
    
    return rss_1 + rss_2

class TreeNode:
    # treenode - automatically trains itself by building the tree beneath itself when initialised in a recursive manner
    def __init__(self, X, y, depth_level, max_depth=3, features_sample=None):
        self.depth_level = depth_level
        self.max_depth   = max_depth
        if features_sample:
            self.features_sample = X.shape[1]
        else:
            self.features_sample = X.shape[1]/3
            
        if depth_level==max_depth or len(y)<=1:
            self.type             = "leaf"
            self.prediction_value = np.mean(y)
            self.y                = y
        else:
            self.type  = "split"
            
            self.split_info = self.create_split(X_create_split=X, y_create_split=y)

            left_indices  = X[:, self.split_info['best_attribute']]<=self.split_info['best_split_value']
            right_indices = X[:, self.split_info['best_attribute']]>self.split_info['best_split_value']
            
            self.X_left = X[left_indices, :]
            self.y_left = y[left_indices]

            self.X_right = X[right_indices, :]
            self.y_right = y[right_indices]
            
            if len(self.y_right)==0 or len(self.y_left)==0:
                self.type             = "leaf"
                self.prediction_value = np.mean(y)
                self.y                = y
            else:
                self.create_lower_branches()
    
    # create lower branches - makes two new tree nodes below itself using the respective subsets of the data
    def create_lower_branches(self):
        self.left  = TreeNode(self.X_left, 
                              self.y_left, 
                              depth_level=self.depth_level+1,
                              max_depth = self.max_depth)
        self.right = TreeNode(self.X_right, 
                              self.y_right, 
                              depth_level=self.depth_level+1,
                              max_depth = self.max_depth)
    
    # searches through the attributes and data to create the split
    def create_split(self, X_create_split, y_create_split):
        min_RSS = np.inf
        size = int(np.round(self.features_sample))
        attributes_considered = np.random.choice(X_create_split.shape[1], size=size, replace=False)
        for attribute in attributes_considered:
            unique_values = np.unique(X_create_split[:, attribute])
            
            for s in unique_values:
                rss = calculate_RSS(X_create_split, y_create_split, attr=attribute, s=s)
                if rss < min_RSS:
                    min_RSS = rss
                    best_attribute = attribute
                    best_split_value = s
        return {'min_RSS': min_RSS, 
                'best_attribute': best_attribute, 
                'best_split_value': best_split_value}
    
    # predicts for a single observation
    # again works recursively, retrieving the prediction value of the relevant branch for each split
    def predict(self,X):
        if self.type=="leaf":

            return self.prediction_value
        else:
            
            if X[self.split_info['best_attribute']] <= self.split_info['best_split_value']:

                return self.left.predict(X)
            else:

                return self.right.predict(X)
    
    # predicts for a group of observation
    def predict_set(self, X):
        return np.array([self.predict(x) for x in X])

class RandomForest:
    # the random forest uses a collection of trees, where each tree sees a boostrap of the data (selection with replacement)
    # and each split sees a random subset of the features
    
    def __init__(self, number_trees=100, data_proportion_per_tree=0.66, max_depth=3):
        self.number_trees = number_trees
        self.data_proportion_per_tree = data_proportion_per_tree
        self.amount_per_tree = int(data_proportion_per_tree*X_train.shape[0])
        self.max_depth = max_depth
    
    # train by making the trees and store them
    def train(self, X, y):
        self.trees = []
        for _ in range(self.number_trees):
            indices_to_send = np.random.choice(X_train.shape[0], size = self.amount_per_tree, replace=True)
            
            # this initialisation of the tree automatically trains it
            tree = TreeNode(X = X_train[indices_to_send], y = y_train[indices_to_send], depth_level=0, max_depth=self.max_depth)
            self.trees.append(tree)
            
    # predict by predicting for each tree, then takng the mean
    def predict(self, X):
        return np.array([np.mean([tree.predict(x) for tree in self.trees]) for x in X])

# function to get MSE quickly
def get_MSE(y_predictions, y_labels):
    return ((y_predictions-y_labels)**2).mean()

def get_MRSE(y_predictions, y_labels):
    return ((y_predictions-y_labels)**2).mean()**0.5

In [35]:
def normalising(data):
    return (data-data.mean(axis=0))/data.std(axis=0)

In [ ]:
column_to_ignore = ["SalePrice"]
target_column     = ["SalePrice"]

X_columns = [column for column in fully_properties_processed_ohe.columns if column not in column_to_ignore]
X = fully_properties_processed_ohe[X_columns].values
y = fully_properties_processed_ohe[target_column].values
y = y/100000

X_normalised = normalising(X)
X_normalised.std(axis=0)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_normalised, y, test_size=0.3, random_state=2022)

In [ ]:
rf = RandomForest(number_trees=100, data_proportion_per_tree=0.7, max_depth=10)
rf.train(X_train, y_train)
predictions = rf.predict(X_train)
train_MSE = get_MSE(predictions, y_train)
train_MRSE = get_MRSE(predictions, y_train)
predictions = rf.predict(X_test)
test_MSE  = get_MSE(predictions, y_test)
test_MRSE  = get_MRSE(predictions, y_test)
print(f"Train MSE: {train_MSE}, Train MRSE: {train_MRSE}")
print(f"Test MSE: {test_MSE}, Test MRSE: {test_MRSE}")

# Lasso

In [39]:
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

In [40]:
# Set up a list of Lasso alphas to check.
l_alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

# Cross-validate over our list of Lasso alphas.
lasso_cv = LassoCV(alphas=l_alphas, cv=5, max_iter=5000)

# Fit model using best ridge alpha!
lasso_model = lasso_cv.fit(Z_train, y_train);

In [41]:
print(f'Train score:', lasso_model.score(Z_train, y_train))
print(f'Test score:', lasso_model.score(Z_test, y_test))

Train score: 0.9064229737975885
Test score: 0.8886827550959064


In [42]:
#Compute predictions on test data
preds_l = lasso_model.predict(Z_test)
log_preds_l = np.log(preds_l)

In [ ]:
plt.scatter(y_test, preds_l)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, preds_l, c='crimson')
p1 = max(max(preds_l), max(y_test))
p2 = min(min(preds_l), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
explainer = shap.explainers.Linear(lasso_cv, Z_test)
shap_values = explainer.shap_values(Z_test)
shap.summary_plot(shap_values, Z_test)

In [ ]:
explainer = shap.explainers.Linear(lasso_cv, Z_test)
shap_values = explainer.shap_values(Z_test)
shap.summary_plot(shap_values, Z_test, max_display=Z_test.shape[1])

#Ridge

In [47]:
r_alpha = np.logspace (0,5,200)

# fits multiple alphas
ridgecv = RidgeCV(alphas = r_alpha, cv = 5)
ridgecv = ridgecv.fit(Z_train, y_train)

print('optimal ridge alpha: ', ridgecv.alpha_)
print('best ridge R2: ', ridgecv.score(Z_train, y_train))

optimal ridge alpha:  2.3816855519761586
best ridge R2:  0.9064137924185774


In [48]:
ridge = Ridge(alpha = ridgecv.alpha_)

In [49]:
ridge_mod = ridge.fit(Z_train, y_train)
# predict on test data
preds_r = ridge_mod.predict(Z_test)
# evaluate model performance
print('ridge test R2: ', ridge_mod.score(Z_test, y_test))
print('ridge test RMSE: ', rmse(y_test, preds_r))

ridge test R2:  0.8887428828153559
ridge test RMSE:  0.268416651372836


In [ ]:
plt.scatter(y_test, preds_r)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, preds_r, c='crimson')
p1 = max(max(preds_r), max(y_test))
p2 = min(min(preds_r), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
explainer = shap.explainers.Linear(ridge_mod, Z_test)
shap_values = explainer.shap_values(Z_test)
shap.summary_plot(shap_values, Z_test)

In [ ]:
explainer = shap.explainers.Linear(ridge_mod, Z_test)
shap_values = explainer.shap_values(Z_test)
shap.summary_plot(shap_values, Z_test, max_display=Z_test.shape[1])

# Elastic Net

In [ ]:
enet_alpha = np.arange(0, 1, 0.005)
enet_ratio = [.01, .1, .2, .3, .5, .7, .9, .95, .99, 1]

# fits multiple alphas and rhos
enetcv = ElasticNetCV(alphas = enet_alpha, l1_ratio = enet_ratio, cv = 5)
enetcv = enetcv.fit(Z_train, y_train)

print('optimal enet alpha: ', enetcv.alpha_)
print('optimal enet lambda: ', enetcv.l1_ratio_)
print('best elastic net R2: ', enetcv.score(Z_train, y_train))

In [55]:
enet = ElasticNet(alpha = enetcv.alpha_, l1_ratio = enetcv.l1_ratio_)

In [56]:
# fit model to train data
enet_mod = enet.fit(Z_train, y_train)
# predict on test data
preds_e = enet_mod.predict(Z_test)
# evaluate model performance
print('elastic net test R2: ', enet_mod.score(Z_test, y_test))
print('elastic net test RMSE: ', rmse(y_test, preds_e))

elastic net test R2:  0.889167645646346
elastic net test RMSE:  0.2679037743123376


In [ ]:
plt.scatter(y_test, preds_e)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, preds_e, c='crimson')
p1 = max(max(preds_e), max(y_test))
p2 = min(min(preds_e), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
explainer = shap.explainers.Linear(enet_mod, Z_test)
shap_values = explainer.shap_values(Z_test)
shap.summary_plot(shap_values, Z_test)

In [ ]:
explainer = shap.explainers.Linear(enet_mod, Z_test)
shap_values = explainer.shap_values(Z_test)
shap.summary_plot(shap_values, Z_test, max_display=Z_test.shape[1])

#Final Visualizations which shows what are the features that affect the sales price

In [61]:
coef_labels = [col for col in fully_properties_processed_ohe.columns if col != 'saleprice']
lasso_coef = pd.DataFrame(lasso_model.coef_)             
lasso_coef = lasso_coef[lasso_coef[0] != 0]                                  
print(f'the model produced {lasso_coef.shape[0]} non-zero coefficients.')
# sort
lasso_coef = lasso_coef.reindex(lasso_coef[0].abs().sort_values(ascending=True).index)

the model produced 63 non-zero coefficients.


In [62]:
# get list of top 20 predictors
predictors = list(lasso_coef.index[-20:])
len(predictors)

20

In [63]:
final=[]
for col in predictors:
    final.append(fully_properties_processed.columns[col])

In [ ]:
final

In [ ]:
subplot_scatter(fully_properties_processed,final)